### 作業目標: 了解N-Gram如何藉由文本計算機率

### 為何需要使用馬可夫假設來簡化語言模型的計算?

原本的語言模型利用貝氏定理計算機率時為:
$$
\begin{aligned}
&W = (W_1W_2W_3W_4…W_m) \\
&P(W_1,W_2,W_3,W_4,…,W_m) = P(W_1)*P(W_2|W_1)*P(W_3|W_1,W_2)*…*P(W_m|W_1,…,W_{m-1})
\end{aligned}
$$

為何需要引入馬可夫假設使機率簡化為:
$$
P(W_m|W_1,W_2,W_3,…,W_{m-1}) = P(W_m|W_{m-n+1},W_{m-n+2},…,W_{m-1})
$$

In [ ]:
'''
因為馬可夫假設主張下一個字詞出現的頻率只依賴前面的一個或數個字詞，而不需要計算（考慮）到太多先前的字詞 
(Markov models...assume we can predict the probability of some future unit without looking too far into the past

Reference: 
1. Jurafsky, Daniel and James H. Martin. "Ch3- N-gram language models." Speech and Language Processing. web.stanford.edu/~jurafsky/slp3/3.pdf. Accessed 
26 Jan 2021.
'''

### 以Bigram模型下判斷語句是否合理

已知的機率值有
1. p(i|_start_) = 0.25
2. p(english|want) = 0.0011
3. p(food|english) = 0.5
4. p(_end_|food) = 0.68
5. P(want|_start_) = 0.25
6. P(english|i) = 0.0011

In [ ]:
import numpy as np
import pandas as pd
words = ['i', 'want', 'to', 'eat', 'chinese', 'food', 'lunch', 'spend']
word_cnts = np.array([2533, 927, 2417, 746, 158, 1093, 341, 278]).reshape(1, -1)
df_word_cnts = pd.DataFrame(word_cnts, columns=words)
df_word_cnts

,i,want,to,eat,chinese,food,lunch,spend
0,2533,927,2417,746,158,1093,341,278


In [ ]:
# 記錄當前字與前一個字詞的存在頻率
bigram_word_cnts = [[5, 827, 0, 9, 0, 0, 0, 2], [2, 0, 608, 1, 6, 6, 5, 1], [2, 0, 4, 686, 2, 0, 6, 211],
                    [0, 0, 2, 0, 16, 2, 42, 0], [1, 0, 0, 0, 0, 82, 1, 0], [15, 0, 15, 0, 1, 4, 0, 0],
                    [2, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0]]

df_bigram_word_cnts = pd.DataFrame(bigram_word_cnts, columns=words, index=words)
df_bigram_word_cnts

,i,want,to,eat,chinese,food,lunch,spend
i,5,827,0,9,0,0,0,2
want,2,0,608,1,6,6,5,1
to,2,0,4,686,2,0,6,211
eat,0,0,2,0,16,2,42,0
chinese,1,0,0,0,0,82,1,0
food,15,0,15,0,1,4,0,0
lunch,2,0,0,0,0,1,0,0
spend,1,0,1,0,0,0,0,0


由上表可知當前一個字詞(列)是want的時候, 當前字詞(行)是to的頻率在文本中共有608次

In [ ]:
#請根據給出的總詞頻(df_word_cnts)與bigram模型的詞頻(df_bigram_word_cnts)計算出所有詞的配對機率(ex:p(want|i))
df_bigram_prob = df_bigram_word_cnts.copy()

df_bigram_prob = np.divide(df_bigram_prob, df_word_cnts.T) #.T --> transpose the array
                #np.divide(df_bigram_prob, df_word_cnts.values.T) #.value --> take only numeric values from array
df_bigram_prob

,i,want,to,eat,chinese,food,lunch,spend
i,0.001974,0.32649,0.000000,0.003553,0.000000,0.000000,0.000000,0.000790
want,0.002157,0.00000,0.655879,0.001079,0.006472,0.006472,0.005394,0.001079
to,0.000827,0.00000,0.001655,0.283823,0.000827,0.000000,0.002482,0.087298
eat,0.000000,0.00000,0.002681,0.000000,0.021448,0.002681,0.056300,0.000000
chinese,0.006329,0.00000,0.000000,0.000000,0.000000,0.518987,0.006329,0.000000
food,0.013724,0.00000,0.013724,0.000000,0.000915,0.003660,0.000000,0.000000
lunch,0.005865,0.00000,0.000000,0.000000,0.000000,0.002933,0.000000,0.000000
spend,0.003597,0.00000,0.003597,0.000000,0.000000,0.000000,0.000000,0.000000


請根據已給的機率與所計算出的機率(df_bigram_prob), 試著判斷下列兩個句子哪個較為合理

s1 = “i want english food”

s2 = "want i english food"

In [ ]:
'''
given that: 
p(i|start) = 0.25
p(english|want) = 0.0011
p(food|english) = 0.5
p(end|food) = 0.68
P(want|start) = 0.25
P(english|i) = 0.0011

the probability of s1 (P(I|<start>) * P(want|I) * P(english|want) * P(food|english)) * p(end|food)) is:
0.25 * 0.326 * 0.0011 * 0.5 * 0.68

the probability of s2 P(want|<start>) * P(I|want) * P(english|I) * P(food|english)) * p(end|food)) is:
0.25 * 0.002 * 0.0011 * 0.5 * 0.68
'''
print("the probability of s1 is approximately: {:.10f}, the probability of s2 is approximately: {:.10f}"
.format(0.25 * df_bigram_prob['want']['i'] * 0.0011 * 0.5 * 0.68, 0.25 * df_bigram_prob['i']['want'] * 0.0011 * 0.5 * 0.68))

###S1 is more reasonable###

the probability of s1 is approximately: 0.0000305268, the probability of s2 is approximately: 0.0000002017
